In [1]:
#Importing sim4life modules
import s4l_v1.document as document
import s4l_v1.model as model
from s4l_v1.model import Vec3

#Creating multitree model groups
TissueGroup = model.CreateGroup('Tissue')
NerveGroup = model.CreateGroup('Nerve Trajectories')

ElectrodeGroup = model.CreateGroup('Electrodes')
CuffGroup = model.CreateGroup('Cuffs')
ContactsGroup = model.CreateGroup('Contacts')
ElectrodeGroup.Add([CuffGroup, ContactsGroup])

#Creating Vagus nerve tissue as cylinder
BasicCylinder = model.CreateSolidCylinder( Vec3(0,0,0), Vec3(0,0,300), 10)
BasicCylinder.Name = 'Vagus Tissue'
TissueGroup.Add(BasicCylinder)

#Adding two identical axons into the nerve
Axon = model.CreatePolyLine([Vec3(0,-4,0), Vec3(0,-4,300)])
Axon.Name = 'Axon_1'
#Axon_2 = Axon.Clone()
#Axon_2.Translation(Vec3(0,4,0))
Axon_2 = model.CreatePolyLine([Vec3(0,4,0), Vec3(0,4,300)])
Axon_2.Name = 'Axon_2'
NerveGroup.Add([Axon, Axon_2])

#Creating double cuff
StimCuff = model.CreateSolidTube(Vec3(0,0,50), Vec3(0,0,20), 12, 10)
StimCuff.Name = 'Stimulating Cuff Electrode'
SenseCuff = model.CreateSolidTube(Vec3(0,0,200), Vec3(0,0,20),12,10)
SenseCuff.Name = 'Sensory Cuff Electrode'
CuffGroup.Add([StimCuff, SenseCuff])

model.Entity.Delete(BasicCylinder) #Deletes Tissue

In [2]:
#Import s4l modules
from cmath import sin
from re import S
import s4l_v1.document as document
import s4l_v1.model as model
import s4l_v1.simulation.emlf as emlf
import s4l_v1.analysis as analysis
import s4l_v1.analysis.viewers as viewers
from s4l_v1.model import Vec3
import s4l_v1.units as units
from s4l_v1 import Unit
import s4l_v1.simulation.neuron as neuron
import s4l_v1.materials.database as database

import random
import numpy as np
import math
import sys, os

#Stimulation Setup
stim = emlf.ElectroQsOhmicSimulation()
stim.Name = 'Stimulation'
stim.SetupSettings.Frequency = 1000

component__plane_x = stim.AllComponents["Plane X+"]
component__plane_x_1 = stim.AllComponents["Plane X-"]
component__background = stim.AllComponents["Background"]
component__plane_y = stim.AllComponents["Plane Y+"]
component__plane_y_1 = stim.AllComponents["Plane Y-"]
component__plane_z = stim.AllComponents["Plane Z+"]
component__plane_z_1 = stim.AllComponents["Plane Z-"]
component__overall_field = stim.AllComponents["Overall Field"]
        
#Boundary Settings
cathode_BC = stim.AddBoundarySettings()
cathode_BC.DirichletValue = 5
cathode_BC.Name = "Cathode"
#stim.Add(cathode_BC, [model.AllEntities()["Stim BC 1"],model.AllEntities()["Stim BC"]])
stim.Add(cathode_BC, model.AllEntities()["Stim BC"])

anode_BC = stim.AddBoundarySettings()
anode_BC.DirichletValue = -5
anode_BC.Name = "Anode"
stim.Add(anode_BC, model.AllEntities()["Stim BC 1"])

# Editing BoundarySettings "Boundary Settings"
boundary_settings = [x for x in stim.AllSettings if isinstance(x, emlf.BoundarySettings) and x.Name == "Boundary Settings"][0]
components = [component__plane_x, component__plane_x_1, component__plane_y, component__plane_y_1, component__plane_z, component__plane_z_1]
stim.Add(boundary_settings, components)
boundary_settings.BoundaryType = boundary_settings.BoundaryType.enum.Flux
print(boundary_settings)

#Materials
Material_Settings = emlf.MaterialSettings()
stim.LinkMaterialWithDatabase(Material_Settings, database["IT'IS LF 4.1"]["Nerve"])
stim.Add(Material_Settings, [model.AllEntities()["Vagus Tissue"], model.AllEntities()["Axon 1"], model.AllEntities()["Axon 2"]])

Material_Settings = emlf.MaterialSettings()
stim.LinkMaterialWithDatabase(Material_Settings, database["Generic 1.1"]["Platinium"])
stim.Add(Material_Settings, [model.AllEntities()["Stim Ring Contact"], model.AllEntities()["Stim Ring Contact 1"], model.AllEntities()["Sense Ring Contact"], model.AllEntities()["Sense Ring Contact 1"]])

Material_Settings = emlf.MaterialSettings()
stim.LinkMaterialWithDatabase(Material_Settings, database["Generic 1.1"]["Polyimide"])
stim.Add(Material_Settings, [model.AllEntities()["Sensory Cuff"],model.AllEntities()["Stimulating Cuff"]])

#GRID SETTINGS
global_grid_settings = stim.GlobalGridSettings
global_grid_settings.DiscretizationMode = global_grid_settings.DiscretizationMode.enum.Manual
global_grid_settings.MaxStep = np.array([1.5, 1.5, 1.5]), units.MilliMeters
global_grid_settings.Resolution = np.array([0.625, 0.625, 0.625]), units.MilliMeters
global_grid_settings.PaddingMode = global_grid_settings.PaddingMode.enum.Manual
global_grid_settings.BottomPadding = np.array([2.0, 2.0, 2.0]), units.MilliMeters
global_grid_settings.TopPadding = np.array([2.0, 2.0, 2.0]), units.MilliMeters

#Removing AutomaticGridSettings Automatic
stim.RemoveSettings([x for x in stim.AllSettings if isinstance(x, emlf.AutomaticGridSettings) and x.Name == "Automatic"][0])

#Removing AutomaticVoxelerSettings Automatic Voxeler Settings
stim.RemoveSettings([x for x in stim.AllSettings if isinstance(x, emlf.AutomaticVoxelerSettings) and x.Name == "Automatic Voxeler Settings"][0])

# Adding a new ManualVoxelerSettings
manual_voxeler_settings = emlf.ManualVoxelerSettings()
manual_voxeler_settings.Name = "Cuffs"
manual_voxeler_settings.Priority = 0
stim.Add(manual_voxeler_settings, [model.AllEntities()["Sensory Cuff"],model.AllEntities()["Stimulating Cuff"]])

manual_voxeler_settings = emlf.ManualVoxelerSettings()
manual_voxeler_settings.Name = "Contacts"
manual_voxeler_settings.Priority = 1
stim.Add(manual_voxeler_settings, [model.AllEntities()["Stim Ring Contact"], model.AllEntities()["Stim Ring Contact 1"], model.AllEntities()["Sense Ring Contact"], model.AllEntities()["Sense Ring Contact 1"]])

manual_voxeler_settings = emlf.ManualVoxelerSettings()
manual_voxeler_settings.Name = "Tissue"
manual_voxeler_settings.Priority = 3
stim.Add(manual_voxeler_settings, [model.AllEntities()["Vagus Tissue"]])

manual_voxeler_settings = emlf.ManualVoxelerSettings()
manual_voxeler_settings.Name = "Axons"
manual_voxeler_settings.Priority = 4
stim.Add(manual_voxeler_settings, [model.AllEntities()["Axon 1"], model.AllEntities()["Axon 2"]])

# Editing SolverSettings "Solver"
solver_settings = stim.SolverSettings
solver_settings.PredefinedTolerances = solver_settings.PredefinedTolerances.enum.High

# Update the materials with the new frequency parameters
stim.UpdateAllMaterials()
                             
document.AllSimulations.Add( stim )
    
sense = emlf.ElectroQsOhmicSimulation.Clone(stim)
sense.Name = 'Sensing'
sense.RemoveSettings([x for x in sense.AllSettings if isinstance(x, emlf.BoundarySettings) and x.Name == "Cathode"][0])
sense.RemoveSettings([x for x in sense.AllSettings if isinstance(x, emlf.BoundarySettings) and x.Name == "Anode"][0])

cathode_BC = sense.AddBoundarySettings()
sense.Add(cathode_BC, model.AllEntities()["Sense BC"])
cathode_BC.DirichletValue = 5
cathode_BC.Name = "Cathode"
                     
anode_BC = sense.AddBoundarySettings()
sense.Add(anode_BC, model.AllEntities()["Sense BC 1"])
anode_BC.DirichletValue = -5
anode_BC.Name = "Anode"

#Adding a new ManualGridSettings
manual_grid_settings = stim.AddManualGridSettings([model.AllEntities()["Vagus Tissue"], model.AllEntities()["Axon 1"], model.AllEntities()["Axon 2"], model.AllEntities()["Sensory Cuff"],model.AllEntities()["Stimulating Cuff"], model.AllEntities()["Stim Ring Contact"], model.AllEntities()["Stim Ring Contact 1"], model.AllEntities()["Sense Ring Contact"], model.AllEntities()["Sense Ring Contact 1"]])
manual_grid_settings.Name = "All"
manual_grid_settings.MaxStep = np.array([1, 1, 1]), units.MilliMeters
manual_grid_settings.Resolution = np.array([1, 1, 1]), units.MilliMeters

manual_voxeler_settings = emlf.ManualVoxelerSettings()
manual_voxeler_settings.Name = "Boundary"
manual_voxeler_settings.Priority = 2
stim.Add(manual_voxeler_settings, [model.AllEntities()["Stim BC"], model.AllEntities()["Stim BC 1"]])

stim.Add( manual_grid_settings, [model.AllEntities()["Stim BC"], model.AllEntities()["Stim BC 1"]] )

manual_grid_settings = sense.AddManualGridSettings([model.AllEntities()["Vagus Tissue"], model.AllEntities()["Axon 1"], model.AllEntities()["Axon 2"], model.AllEntities()["Sensory Cuff"],model.AllEntities()["Stimulating Cuff"], model.AllEntities()["Stim Ring Contact"], model.AllEntities()["Stim Ring Contact 1"], model.AllEntities()["Sense Ring Contact"], model.AllEntities()["Sense Ring Contact 1"]])
sense.Add( manual_grid_settings, [model.AllEntities()["Sense BC"], model.AllEntities()["Sense BC 1"]] )

manual_voxeler_settings = emlf.ManualVoxelerSettings()
manual_voxeler_settings.Name = "Boundary"
manual_voxeler_settings.Priority = 2
sense.Add(manual_voxeler_settings, [model.AllEntities()["Sense BC"], model.AllEntities()["Sense BC 1"]])

# Update the grid with the new parameters and create voxels
stim.UpdateGrid()
sense.UpdateGrid()
stim.CreateVoxels()
sense.CreateVoxels()

document.AllSimulations.Add( sense )

#Run EM Sims
stim.RunSimulation()
sense.RunSimulation()

<'Boundary Settings' @ s4l_v1.simulation._emlf_shared_impl.BoundarySettings object at 0x7f81340ef550>


In [5]:
#Import s4l modules
from cmath import sin
from re import S
import s4l_v1.document as document
import s4l_v1.model as model
import s4l_v1.simulation.emlf as emlf
import s4l_v1.analysis as analysis
import s4l_v1.analysis.viewers as viewers
from s4l_v1.model import Vec3
import s4l_v1.units as units
from s4l_v1 import Unit
import s4l_v1.simulation.neuron as neuron
import s4l_v1.materials.database as database

import random
import numpy as np
import math
import sys, os

neuro = neuron.Simulation()

to_be_discrete_axon = [model.AllEntities()["Axon 1"], model.AllEntities()["Axon 2"]]
model_properties = model.SennNeuronProperties()
model_properties.AxonDiameter = 10
discretized_axons = model.CreateAxonNeuron(model.AllEntities()["Axon 1"], model_properties)
discretized_axons = model.CreateAxonNeuron(model.AllEntities()["Axon 2"], model_properties)
#NerveGroup.Add(discretized_axons)

setup_settings = neuro.SetupSettings
setup_settings.PerformTitration = True
setup_settings.DepolarizationDetection = setup_settings.DepolarizationDetection.enum.Threshold
setup_settings.DepolarizationThreshold = 80

# Neuron Settings
#neuron_settings = neuron.AutomaticAxonNeuronSettings()
#neuro.Add(neuron_settings,to_be_discrete_axon)

#Link Neuron to Stimulation simulation
link = document.AllSimulations["Stimulation"]
nsource= neuro.AddSource(link, "Overall Field")
#nsource.PulseType = nsource.PulseType.enum.Bipolar
nsource.PulseType = 1
nsource.Interval = 0.1

linesensor = neuro.AddOverallLineSensor()
linesensor.RecordScaledIMembrane = True
linesensor.RecordV = True
linesensor.NumberOfSnapshots = 200

axon_nodes = neuro.GetSectionNames(model.AllEntities()["Axon 2 [SENN Neuron 10.00um]"])
#print(axon_nodes)
point_sensor = neuro.AddPointSensor(model.AllEntities()["Axon 2 [SENN Neuron 10.00um]"])
point_sensor.SectionName = "node[50]"
point_sensor.RecordV= True
point_sensor.RelativePosition = 0.5

axon_nodes = neuro.GetSectionNames(model.AllEntities()["Axon 1 [SENN Neuron 10.00um]"])
#print(axon_nodes)
point_sensor = neuro.AddPointSensor(model.AllEntities()["Axon 1 [SENN Neuron 10.00um]"])
point_sensor.SectionName = "node[50]"
point_sensor.RecordV= True
point_sensor.RelativePosition = 0.5

# Editing SolverSettings "Solver"
solver_settings = neuro.SolverSettings
solver_settings.Duration = 3.5, units.MilliSeconds
solver_settings.TimeStep=0.0025 , units.MilliSecond
solver_settings.NumberOfThreads = 8

#for i in range(6):
  
document.AllSimulations.Add( neuro )
neuro.RunSimulation()